<a href="https://colab.research.google.com/github/brian0714/ATCiCu_transport/blob/main/NLP%E8%A8%98%E4%BA%8B%E8%B3%87%E6%96%99%E5%95%8F%E9%A1%8C%E5%88%86%E9%A1%9EAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLP記事自動分類API**

In [ ]:
# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager

# 改style要在改font之前
# plt.style.use('seaborn')

fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

--2023-08-08 07:23:18--  https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.google.com (drive.google.com)... 173.194.215.102, 173.194.215.113, 173.194.215.100, ...
Connecting to drive.google.com (drive.google.com)|173.194.215.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8rlsuovue2ornqvefa87rg0vem5nr969/1691479350000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_?uuid=255d557f-85b7-43f1-9540-70eba951c9cc [following]
--2023-08-08 07:23:22--  https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8rlsuovue2ornqvefa87rg0vem5nr969/1691479350000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_?uuid=255d557f-85b7-43f1-9540-70eba951c9cc
Resolving doc-0k-9o-docs.googleusercontent.com (doc-0k-9o-docs.googleusercontent.com)... 74.125.141.132, 2607:f8b0:400c:c06::84
Connecting to do

In [ ]:
#fasttext安裝 / 實作：https://blog.csdn.net/vivian_ll/article/details/10621582
!pip install fastText

#jieba安裝 多類別預測用模型
!pip install https://github.com/APCLab/jieba-tw/archive/master.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fastText: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199773 sha256=0050810c37651c6dc69f8af08ba642b9a390296484b18f2d89e12fadbdeb8ed4
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fastText
     \ 6.6 MB 4.6 MB/s 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for jieba: filename=jieba-0.39.1-py3-none-any.whl size=6603857 sha256=a662c744aa2dfc0170aa2e217f799fd0b7bd37938733dd3b829297406e60951f
  Stored in directory: /tmp/pip-ephem-wheel-cache-n68jdauy/wheels/78/ec/55/111762988bb87acfe13b47ed57d09c643a4239640c6b833bb2
Successfully built jieba
  Attempting uninstall: jieba
    Found existing installation: jieba 0.42.1
    Uninstalling jieba-0.42.1:
      Successfully uninstalled jieba-0

In [ ]:
import openpyxl
import pandas as pd
import numpy as np
import jieba
import re
import random
import os
import fasttext
from datetime import datetime,timezone,timedelta
from collections import Counter
from openpyxl import load_workbook
from openpyxl.drawing.image import Image

In [ ]:
#NLP記事自動分類(訓練模型)

#所有檔案雲端位址
cloud = True
if cloud:
  cloud_path = '/content/drive/MyDrive/T大使 python實作內容/NLP叡揚實作/NLP記事自動分類API'

#載入資料
def load_raw_data(raw_data_file_path, sheet_name = 'Sheet0'):
  # raw_data_file_path = f'{cloud_path}/input_training_datas/{file_name}'
  df = pd.read_excel(raw_data_file_path, sheet_name = sheet_name)
  df = df.dropna( subset=['記事類別'] ).reset_index(drop=True)
  df = df.loc[:, ['記事內容', '記事類別']]
  print(f'原始資料數： {len(df)}')
  return df
# df = load_raw_data('2022年所有記事資料(raw data).xlsx')
df = load_raw_data(raw_data_file_path = f'{cloud_path}/input_training_datas/2022年所有記事資料(raw data).xlsx')

#取得記事分類類別及分類關鍵字
def get_categories_relation(categories_relation_file):
  # 載入資料
  raw_data_file_path = f'{cloud_path}/input_training_datas/{categories_relation_file}'
  categories_relation = pd.read_excel(raw_data_file_path)
  # 讀取 Excel 檔案，並將空值轉成N/A
  categories_relation = categories_relation.fillna("N/A")

  product_name = categories_relation['產品類別'][0]
  CATE = categories_relation['記事分類類別'].values.tolist()
  cate_dict = {}
  for cate in CATE:
    cate_dict[cate] = [cate]
  for cate, relation in zip(categories_relation['記事分類類別'], categories_relation['關鍵字(以;做分隔)']):
    if relation == 'N/A':
      pass
    elif ';' in relation:
      relations = relation.split(';')
      if relations[-1] == '':
        relations = relations[:-1]
      cate_dict[cate] += relations
    else:
      cate_dict[cate] += [relation]
  return product_name, CATE, cate_dict
# product_name, CATE, cate_dict = get_categories_relation(categories_relation_file = 'CRM記事類別關聯.xlsx')
product_name, CATE, cate_dict = get_categories_relation(categories_relation_file = 'OD記事類別關聯.xlsx')

#篩選記事類別
#prod_cates_relation 為產品相關的連結關係
prod_cates_relation = {
    'CRM':['CRM','Insight'],
    'Finance':['Finance','General Ledger'],
    'Bizform':['Bizform'],
    'OD':['Official Document'],
    'HCM':['HCM'],
    'Netzero':['Netzero'],
    'TTC':['TTC'],
    'Knowledge':['Knowledge'],
}

def df_categories_filter(df, product_name):
  #判斷產品相關的其他產品詞
  prod_cates = prod_cates_relation[product_name]
  # 建立一個空的 DataFrame filter_df
  filter_df = pd.DataFrame(columns=df.columns)
  # 將符合條件的資料新增至 filter_df
  if len(prod_cates) == 1:
    conditions = df['記事類別'].apply(lambda categories: prod_cates[0] in categories)
  elif len(prod_cates) == 2:
    conditions = (df['記事類別'].apply(lambda categories: prod_cates[0] in categories) | df['記事類別'].apply(lambda categories: prod_cates[1] in categories))
  filter_df = df[conditions]
  # 重設索引
  filter_df = filter_df.reset_index(drop=True)
  return filter_df
filter_df = df_categories_filter(df, product_name = product_name)

#關鍵字標註做訓練
def keyword_labeling(filter_df, CATE, cate_dict): #CATE表分類種類
  #保留指定欄位 排除掉其他欄位
  imp_columns = ['記事內容',]
  filter_df = filter_df[imp_columns]
  # 將DataFrame 中的所有 NaN 值替換为空字串
  filter_df = filter_df.fillna(value = '')

  #Y表預測分類
  Y = [ [] for i in range(len(filter_df))]
  print(f'{product_name}記事分類種類： {len(CATE)}')

  #篩選記事內容是否有對應到之關鍵字
  for i in range(len(filter_df)):
    for cates in cate_dict:
      for cate in cate_dict[cates]:
        if cate in filter_df['記事內容'][i]:
          Y[i].append(cates)
          break

  print(f'{product_name}原始資料數: {len(filter_df)}')
  filter_df['Y'] = Y

  #選出有關鍵字標註的資料
  filter_df = filter_df[filter_df['Y'].apply(lambda x: len(x) > 0)]
  print('filter_df(可訓練)資料數:',len(filter_df))
  return filter_df
filter_df = keyword_labeling(filter_df, CATE = CATE, cate_dict = cate_dict)
print('='*200)

#取得停用詞檔案
def get_stopwords_list(file_path):
  with open(file_path,'r',encoding = 'utf-8') as f:
    stopwords_list = [word.strip('\n') for word in f.readlines()]
  return stopwords_list

#資料處理至執行模型
def preprocessing_to_fasttext_model(df, preprocessed_file_name, product_name, CATE, word_cut = 'uni', wordNgrams = 2, model_threshold = 0.001, epoch = 100, lr = 0.8, train_test_split = False, save = False):
  df = df.reset_index(drop = True)
  print(df)
  print('*'*200)
  #取得停用詞
  stop_words = get_stopwords_list(file_path = f'{cloud_path}/input_training_datas/stopwords.txt')

  #處理文本資料
  # print(df['記事內容'])
  notes = df['記事內容'].values.tolist()
  print('可訓練記事內容大小：', len(notes))
  Y = df['Y']

  #需將y_label轉換成數值對照
  y_labels_lists = CATE
  #轉換成 -> y_labels = {'社群媒體(LINE@)':1, '社群媒體(Facebook)':2,.....}
  y_labels = {}
  for i, label in enumerate(y_labels_lists):
    y_labels[label] = i+1
  # print(y_labels)

  #資料預處理+儲存至txt供訓練
  sentences = preprocess_text_ova(content_lines = notes, Y = Y, y_labels = y_labels, stop_words = stop_words,
                    word_cut = word_cut, save_text = True, preprocessed_file_name = preprocessed_file_name)
  #可能會因jieba無法處理而資料減少
  print('處理後之句子資料數：', len(sentences), '\n')

  #決定執行模型資料
  #執行訓練測試資料切分
  if train_test_split:
    test_size = 0.2
    input_path = f'{cloud_path}/API_temp_data/{product_name}_temp_data/{preprocessed_file_name}'
    input_path_train = prepend_file_name(input_path, 'train')
    input_path_test = prepend_file_name(input_path, 'test')
    train_test_split_file(input_path, input_path_train, input_path_test, test_size = 0.2, random_state = random.randint(0, 1000), encoding = 'utf-8')
  #預設無測試
  else:
    input_path_train = f'{cloud_path}/API_temp_data/{product_name}_temp_data/{preprocessed_file_name}'
    input_path_test = f'{cloud_path}/API_temp_data/{product_name}_temp_data/{preprocessed_file_name}'

  #建立模型
  classifier = fasttext.train_supervised(input = input_path_train, dim = 100, epoch = epoch,
                        bucket = 200000, lr = lr, wordNgrams = wordNgrams, loss = 'ova',
                        t = model_threshold) #lr, epoch可調高 // 單字元wordNgrams可調成3 // t表threshold
  #儲存model
  #取得時間作為模型版本
  timecode = get_current_datetime()
  model_name = f'{product_name}_fasttext_classifier_V{timecode}.model'
  if save:
    classifier.save_model(f'{cloud_path}/product_models/{product_name}_class_model/{model_name}')

  result = classifier.test(input_path_test)

  def print_results(N, p, r):
      print("資料集大小\t" + str(N))
      print("P@{}(Precision)\t{:.3f}".format(1, p))
      print("R@{}(Recall)\t{:.3f}".format(1, r))
      f1_score = 2 * (p * r) / (p + r)
      print("F1@{}(F1 score)\t{:.3f}".format(1, f1_score))

  print_results(*classifier.test(input_path_test))
  print('*'*200)

#資料預處理+儲存至txt供訓練
def preprocess_text_ova(content_lines, word_cut, stop_words = [], preprocessed_file_name = [], Y = [], y_labels = {}, save_text = True):
  sentences = []

  #使用繁體詞庫
  # jieba.set_dictionary('dict.txt.big')

  for i, line in enumerate(content_lines):
    try:
      #單字元切割
      if word_cut == 'uni':
        segments = jieba.cut(line, cut_all=False)
        # 建立正規表達式，用於匹配數字和英文字母單詞
        pattern = re.compile(r'[\d]+|[a-zA-Z]+')
        # 逐個詞進行處理
        texts = []
        for segment in segments:
          # 過濾停用詞
          if segment in stop_words:
            continue
          # 判斷是否為英文字母單詞
          elif re.match(pattern, segment):
            texts.append(segment)
          # 對中文詞進行分割為個別字元
          else:
            texts.extend(list(segment))
        segs = ' '.join(texts).replace('\n', '')

      #多字元切割
      elif word_cut == 'vocabulary':
        segs = jieba.lcut(line)
        segs = list(filter(lambda x:len(x) > 1, segs))
        segs = list(filter(lambda x:x not in stop_words, segs))
        segs = ' '.join(segs)

      #在segs前加上y_label
      sentence = ''
      if save_text:
        #將input句子處理成 __label__ 類別 __label__ 類別 詞語 詞語 詞語..之形式之txt
        for y in Y[i]:
          sentence += f'__label__{y_labels[y]} '
      sentences.append(sentence + segs)
        # print('成功轉換：\n')
        # print(sentences + ' '.join(segs))

    except Exception as e:
      print('*'*200)
      print('未轉換文字：', line)
      print('未轉換資料之Y:', Y[i])
      continue
  #將資料儲存至txt
  if save_text:
    output = open(f'{cloud_path}/API_temp_data/{product_name}_temp_data/{preprocessed_file_name}','w',encoding='utf-8')
    for sentence in sentences:
      output.write(sentence + '\n')
  return sentences

#檔案切分訓練測試集
def train_test_split_file(input_path: str,
              output_path_train: str,
              output_path_test: str,
              test_size: float,
              random_state: int=1234,
              encoding: str='utf-8',
              verbose: bool=True):

  random.seed(random_state)

  # we record the number of data in the training and test
  count_train = 0
  count_test = 0
  train_range = 1 - test_size

  with open(input_path, encoding=encoding) as f_in, \
        open(output_path_train, 'w', encoding=encoding) as f_train, \
        open(output_path_test, 'w', encoding=encoding) as f_test:

    for line in f_in:
      random_num = random.random()
      if random_num < train_range:
        f_train.write(line)
        count_train += 1
      else:
        f_test.write(line)
        count_test += 1

  if verbose:
    print('train size: ', count_train)
    print('test size: ', count_test)
    # print('train path: ', input_path_train)
    # print('test path: ', input_path_test)

#輸寫檔案路徑
def prepend_file_name(path: str, name: str) -> str:
  directory = os.path.dirname(path)
  file_name = os.path.basename(path)
  return os.path.join(directory, name + '_' + file_name)

def get_current_datetime():
  dt1 = datetime.utcnow().replace(tzinfo=timezone.utc)
  dt2 = dt1.astimezone(timezone(timedelta(hours=8))) # 轉換時區 -> 東八區
  timecode = dt2.strftime("%Y%m%d%H%M") # 將時間轉換為 string
  return timecode

#FastText(單字元切割) + 單多類別混合資料(訓練測試切分)
preprocessing_to_fasttext_model(df = filter_df, preprocessed_file_name = 'ova_train_data.txt',
                product_name = product_name, CATE = CATE, word_cut = 'vocabulary', wordNgrams = 2, model_threshold = 0.6, epoch = 300, lr = 0.1, train_test_split = True, save = True)


原始資料數： 10775
OD記事分類種類： 7
OD原始資料數: 2781
filter_df(可訓練)資料數: 1622
                                                   記事內容             Y
0                Q:電子收文登錄那邊的刪除怎麼不見?\nA:這兩天版更，按鈕改掉左上角囉\n          [收文]
1     Q:我的帳號m01041027@gmail.com要登入時都會出現「執行時發生錯誤」，無法正...        [系統管理]
2     Q:有份公文(112D000027)從一大早送發文後就一直沒有出現在電子交換平台，發文清單也...  [發文, 電子交換平台]
3     Q:公文112D000399已送發文一個小時但一直沒有出現在電子交換平台，發文資訊顯示「已發...  [發文, 電子交換平台]
4       Q:為什麼電子收文登陸的「刪除」按鈕不見了? 那我以後要怎麼刪除?\nA:按鈕位置已調整到上方    [收文, 公文製作]
...                                                 ...           ...
1617  Q:請問用此公文系統需要固定IP嗎?\nA:不用喔\n\nQ:不用固定IP可以直接跟公部門收...      [收文, 發文]
1618  Q:請教函稿送上去後主管決定不發文該怎麼送歸檔呢?\nA:請問目前公文的狀態是已經決行並且在...      [發文, 檔管]
1619  Q:你好我想請問如果同仁原本在A單位填寫的公文，單位在A，但是後來移轉到B單位之後才公文才送...          [檔管]
1620  Q:公文沒有按鈕可以按也打不開，我看流程現在在我身上，但我都沒看到公文也不能處理\nA:綜合...      [發文, 檔管]
1621  Q:請問一下，TCA現在在使用睿揚公文系統的時候，有人一直出現LOADING的畫面，有的人登...        [系統管理]

[1622 rows x 2 columns]
************************************************************************

In [ ]:
#NLP記事分類分析輸入輸出

#載入指定模型
def load_pretrained_model(categories_relation_file, model_version):
  #獲取所有記事分類類別
  product_name, CATE, cate_dict = get_categories_relation(categories_relation_file = categories_relation_file)
  y_labels_lists = CATE
  #類別編碼inverse
  #轉換成 -> label_to_cate = {'__label__1': '社群媒體(LINE@)', '__label__2': '社群媒體(Facebook)', '__label__3': '行事曆',.....}
  label_to_cate = {}
  for i, label in enumerate(y_labels_lists):
    label_to_cate[f'__label__{i+1}'] = label
  # print('類別對應之編號', label_to_cate, '\n')

  #呼叫模型
  fasttext.FastText.eprint = lambda x: None #避免出現錯誤: Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.
  classifier = fasttext.load_model(f'{cloud_path}/product_models/{product_name}_class_model/{model_version}')
  return classifier, label_to_cate
# classifier = load_pretrained_model(categories_relation_file = 'CRM記事類別關聯.xlsx', model_version = 'CRM_fasttext_classifier_V202308011653.model')
classifier, label_to_cate = load_pretrained_model(categories_relation_file = 'OD記事類別關聯.xlsx', model_version = 'OD_fasttext_classifier_V202308070701.model')

#欲預測之資料處理
def analysis_data_preprocessing():
  # df = load_raw_data(raw_data_file_path = f'{cloud_path}/input_analysis_data/2022年所有記事資料(raw data).xlsx')
  df = load_raw_data(raw_data_file_path = f'{cloud_path}/input_analysis_data/記事資料20230524-20230531.xlsx', sheet_name = '客戶記事')

  # product_name, CATE, cate_dict = get_categories_relation(categories_relation_file = 'CRM記事類別關聯.xlsx')
  product_name, CATE, cate_dict = get_categories_relation(categories_relation_file = 'OD記事類別關聯.xlsx')

  filter_df = df_categories_filter(df, product_name = product_name)
  filter_df = keyword_labeling(filter_df, CATE = CATE, cate_dict = cate_dict)
  stop_words = get_stopwords_list(file_path = f'{cloud_path}/input_training_datas/stopwords.txt')
  sentences = preprocess_text_ova(content_lines = filter_df['記事內容'], stop_words = stop_words, word_cut = 'vocabulary', save_text = False)
  print(len(sentences))
  print(sentences)
  filter_df['預處理之記事'] = sentences
  output_df = filter_df.rename(columns={'Y': '標註關鍵字之分類'})
  return output_df
output_df = analysis_data_preprocessing()

#預測預處理之文本
def data_classification(output_df, label_to_cate):
  all_pred_labels = []
  for texts in output_df['預處理之記事']:
    labels = classifier.predict(texts, k = -1, threshold = 0.1)
    pred_dict = {}
    for label, pb in zip(labels[0], labels[1]):
      pred_dict[label_to_cate[label]] = pb
    pred_labels = ';'.join(list(pred_dict.keys()))
    all_pred_labels.append(pred_labels)
  print(all_pred_labels)
  return all_pred_labels
all_pred_labels = data_classification(output_df, label_to_cate)
output_df['預測分類'] = all_pred_labels

# 儲存 DataFrame 至 Excel 檔案
time_code = get_current_datetime()
output_file_path = f'{cloud_path}/output_analysis_data/NLP記事分析結果_V{time_code}.xlsx'
output_df.to_excel(output_file_path, index=False)

def labels_caculating(all_pred_labels):
  #整理df內所有預測分類
  labels_stat = []
  for labels in all_pred_labels:
    if ';' in labels:
      labels = labels.split(';')
    else:
      labels = [labels]
    labels_stat += labels
  # 使用 map 和 lambda 函數將 '' 取代成 '無法判斷分類'
  labels_stat = list(map(lambda x: '「無法判斷分類」' if x == '' else x, labels_stat))

  #計算labels的種類及其數量
  label_counts = Counter(labels_stat)

  # 排序 categories 和 frequencies，使用 lambda 函式根據 frequencies 進行排序
  sorted_items = sorted(label_counts.items(), key=lambda x: x[1], reverse=True)
  # 解構賦值取出排序後的 categories 和 frequencies
  categories, frequencies = zip(*sorted_items)
  return categories, frequencies

def save_work_book(output_file_path, image_path, save_location):
  # 打開既有的 Excel 檔案
  wb = load_workbook(output_file_path)
  # 載入圖片
  img = Image(image_path)
  # 設定圖片插入位置（插入到單元格(save_location) ex:A1）
  wb.active.add_image(img, save_location)
  # 儲存 Excel 檔案
  wb.save(output_file_path)

#繪製分類類別水平直方圖
def draw_barh_plt(output_file_path, save = True, row = 15, column = 7):
  # #讀取Excel文件並刪除行索引
  # df = pd.read_excel(output_file_path).reset_index(drop=True)
  # all_pred_labels = df['預測分類'].values.tolist()

  #整理df內所有預測分類
  categories, frequencies = labels_caculating(all_pred_labels)

  #繪製barh圖
  plt.figure(figsize=(row, column))
  barh = plt.barh(categories,
      frequencies,
      height = 0.8,
      color='pink')

  #給予各個bars的y值當圖標
  def createLabels(data): # 自定義函數，為引用plusone團隊
    for item in data:
      width = item.get_width() #取得bar x值
      plt.text(
          width*0.85,
          item.get_y(),
          f'{width}',
          ha = "center",
          va = "bottom",
          color = 'black')

  createLabels(barh)
  plt.ylabel("資料數")
  plt.title('分類類別水平直方圖')

  # 儲存圖片到指定路徑
  time_code = get_current_datetime()
  if save:
    image_path = f'{cloud_path}/API_temp_data/pic_temp_data/pie_chart_V{time_code}.png'
    plt.savefig(image_path)
    # 關閉圖形，釋放記憶體
    plt.close()
    save_work_book(output_file_path, image_path, save_location = 'K1')

#繪製分類類別圓餅圖
def draw_pie_plt(output_file_path, save = True, row=12, column=12):
  # #讀取Excel文件並刪除行索引
  # df = pd.read_excel(output_file_path).reset_index(drop=True)
  # all_pred_labels = df['預測分類']

  #整理df內所有預測分類
  categories, frequencies = labels_caculating(all_pred_labels)

  #繪製圓餅圖
  plt.figure(figsize=(row, column))
  def func(s,d):
    #%數太低 圖上展示會字糊
    if s < 2:
      return
    else:
      t = int(round(s/100.*sum(d))) # 透過百分比反推原本的數值
      return f'占比：{s:.1f}%\n資料數：{t}'
  plt.pie(frequencies,
      radius = 1.5,
      labels = categories,
      autopct=lambda i: func(i, frequencies))
  # plt.title('分類類別圓餅圖') #易擋到圖

  # 儲存圖片到指定路徑
  time_code = get_current_datetime()
  if save:
    image_path = f'{cloud_path}/API_temp_data/pic_temp_data/pie_chart_V{time_code}.png'
    plt.savefig(image_path)
    # 關閉圖形，釋放記憶體
    plt.close()
    save_work_book(output_file_path, image_path, save_location = 'K50')


draw_barh_plt(output_file_path, save = True, row=15, column=7)
draw_pie_plt(output_file_path, save = True, row=9, column=5)


原始資料數： 97
OD記事分類種類： 7
OD原始資料數: 97
filter_df(可訓練)資料數: 59
59
['電子 收文 登錄 刪除 天版 按鈕 改掉 左上角', '帳號 m01041027 gmail com 登入 執行 發生 錯誤 登入 產品 修正 電話 確認 登入', '公文 112D000027 一大早 發文 電子 交換 平台 發文 清單 紀錄 產品 修正 回電 確認 客戶 發文', '公文 112D000399 發文 小時 電子 交換 平台 發文 資訊 顯示 發至 前置 產品 修正 回電 確認 客戶 發文', '電子 收文 登陸 刪除 按鈕 不見了 刪除 按鈕 位置 調整 上方', '您好 目前 電子 公文 系統 昨天 昨天 15 20 發生 分鐘 恢復 主任 回報 今早 47 發生 圓圈圈 數字 要點 功能 存取 權限 訊息 主任 回報 39 發生 想要 分文 底下 組長 指定 承辦人員 作業 失敗 訊息 發生 分鐘 恢復 同仁 回報 50 發生 圓圈圈 數字 要點 功能 存取 權限 訊息 畫面 50 轉知 系統 人員 查看 需要 等候 一段 時間 OK 到時 請您 今日 聊天 紀錄 發生 這裡 回報 &# 128578 OK 收到 系統 人員 回覆 喔喔 目前 畫面 恢復 操作 登入 按鍵 深底 權限 訊息 查看 請問 只有 學校 狀況 因應 版本 更新 功能 存取 權限 Vital Official Document 公文 系統 今日 2023 05 24 上午 11 11 緊急 系統 維護 維護 期間 系統 暫時 停止 服務 OK 版本 更新 導致 昨天 下午 發生 狀況 定期 下班 定期 版本 更新 目前 因應 修復 目前 收到 系統 人員 回覆 查明 OK 1110 通知 同仁 重新 登入 試試看 恢復 目前 顯示 功能 存取 權限 感謝 回報 如果 話請 告訴 OK 同仁 目前 登出 剛剛 試了 沒辦法 確認 登入 畫面 首頁 等於 刷新 畫面 清除 cookie 關閉 瀏覽器 重新 打開 試試看 清掉 COOKIE 關掉 瀏覽器 重新 登入 沒辦法 登出 刷新 畫面 轉知 系統 人員 OK 到時候 麻煩 今日 聊天 紀錄 發生 原因 郵件 回報 感謝 整理 OK &# 128578 您好 登出 再一次 執行 cookie 關掉 